In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [2]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime


import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}_abi.json") as f:
        data = json.load(f)
        return data

In [3]:
VAULT_CONTRACT = Web3.to_checksum_address('0x4a10C31b642866d3A3Df2268cEcD2c5B14600523')

In [4]:

# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine, text
from sqlmodel import Session
from core import constants
from core.db import engine
from datetime import datetime, timedelta

# Create a session
session = Session(engine)

# Define the SQL query to get initiated withdrawals without completions
query = text(
    """
    WITH latest_initiated_withdrawals AS (
        SELECT 
            id,
            from_address,
            to_address,
            tx_hash,
            timestamp,
            input,
            block_number,
            ROW_NUMBER() OVER (PARTITION BY from_address ORDER BY timestamp DESC) AS rn
        FROM public.onchain_transaction_history
        WHERE method_id IN (:withdraw_method_id_1)
        AND to_address = ANY(:vault_addresses)
        AND timestamp >= :start_ts
        AND timestamp <= :end_ts
    ),
    has_later_completion AS (
        SELECT DISTINCT 
            i.from_address,
            i.tx_hash
        FROM latest_initiated_withdrawals i
        WHERE i.rn = 1
        AND EXISTS (
            SELECT 1
            FROM public.onchain_transaction_history c
            WHERE c.from_address = i.from_address
            AND to_address = ANY(:vault_addresses)
            AND c.method_id = :complete_method_id
            AND c.timestamp > i.timestamp
        )
    )
    SELECT *
    FROM latest_initiated_withdrawals i
    WHERE i.rn = 1
    AND NOT EXISTS (
        SELECT 1 
        FROM has_later_completion h 
        WHERE h.from_address = i.from_address
    );
    """
)

# Define parameters for the query
params = {
    "withdraw_method_id_1": "0x12edde5e",  # First initiate withdrawal method ID
    "complete_method_id": "0x4f0cb5f3",     # Complete withdrawal method ID
    "vault_addresses": [VAULT_CONTRACT.lower()],
    "start_ts": int((datetime.now() - timedelta(days=30)).timestamp()),  # Start timestamp is last 7 days
    "end_ts": int(datetime.now().timestamp())  # End timestamp is today
}

# Execute the query and fetch results
with session.begin():
    init_withdraws = session.execute(query, params).all()

# print(init_withdraws)
# Create a DataFrame from the results
# Create a DataFrame from the results
df_withdrawals = pd.DataFrame(init_withdraws, columns=["id", "from_address", "to_address", "tx_hash", "timestamp", "input", "block_number", "rn"])

# Display the DataFrame
df_withdrawals.head()

,id,from_address,to_address,tx_hash,timestamp,input,block_number,rn
0,38d3dafc-9fa9-48e1-a887-459392708ee8,0x3cedd3d595bc2eeb59222f6d677971f828fcb7fa,0x4a10c31b642866d3a3df2268cecd2c5b14600523,0x5e593321c640faaf4c5f2a68c5725069ded7303f6843...,1735039805,0x12edde5e000000000000000000000000000000000000...,288097962,1
1,74f76913-1e20-4120-82c0-6f284dbcef63,0x6aa04e484e4e27d8466ec1e73df4c00997ff7e7d,0x4a10c31b642866d3a3df2268cecd2c5b14600523,0x2c12b1678dbe8e9751503525af321a9a8a48600ee83e...,1735383722,0x12edde5e000000000000000000000000000000000000...,289468583,1
2,5f5e74f5-9d1e-440f-8b21-e40d4a3b7076,0x750630450e37a12ccbfb64eaa2d6d1b0318a1316,0x4a10c31b642866d3a3df2268cecd2c5b14600523,0xaa801e757fc4e3cd7e87cf4048f7cd8fd79a0a4a3e9a...,1734423852,0x12edde5e000000000000000000000000000000000000...,285642434,1
3,10fe0131-6cef-4dce-8ead-5cd20c9baec4,0x7d1b001b69d64d782dd3a795f8a276c46a733995,0x4a10c31b642866d3a3df2268cecd2c5b14600523,0x1bdbdbaf1ce65e626e86fec2545abcd45da846c64e4c...,1735557817,0x12edde5e000000000000000000000000000000000000...,290162130,1
4,ff939ed3-02d3-4829-b854-ca71e83656ef,0x8e6f12fcb32b19c5237e29a9fc4436ed0c8cf152,0x4a10c31b642866d3a3df2268cecd2c5b14600523,0xe39f2b0b0e46e1b5eca91dc5a906eb71a5b880523360...,1733475082,0x12edde5e000000000000000000000000000000000000...,281887550,1


In [5]:
w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

rockonyx_delta_neutral_vault_abi = read_abi("rockonyxdeltaneutralvault")
vault_contract = w3.eth.contract(
    address=VAULT_CONTRACT,
    abi=rockonyx_delta_neutral_vault_abi,
)


In [6]:
def _extract_input(data):
    shares = int(data[10:], 16) / 1e6
    return shares

def get_pps(vault_contract: Contract, decimals = 1e6, block_number=200199390):
    try:
        tvl = vault_contract.functions.pricePerShare().call(block_identifier=block_number)

        return tvl / decimals
    except:
        print('block number', block_number)
        return 0

df_withdrawals['shares'] = df_withdrawals.apply(
    lambda row: _extract_input(
        row['input']
    ),
    axis=1
)

df_withdrawals['pps'] = df_withdrawals.apply(
    lambda row: get_pps(
        vault_contract,
        block_number=int(row['block_number'])
    ),
    axis=1
)

df_withdrawals['amount_usd'] = df_withdrawals['shares'] * df_withdrawals['pps']

df_withdrawals[['from_address', 'shares', 'pps', 'amount_usd', 'block_number']]

,from_address,shares,pps,amount_usd,block_number
0,0x3cedd3d595bc2eeb59222f6d677971f828fcb7fa,944.728087,1.060231,1001.630004,288097962
1,0x6aa04e484e4e27d8466ec1e73df4c00997ff7e7d,133.267316,1.061098,141.409682,289468583
2,0x750630450e37a12ccbfb64eaa2d6d1b0318a1316,236.000000,1.059459,250.032324,285642434
3,0x7d1b001b69d64d782dd3a795f8a276c46a733995,3202.507751,1.061668,3399.999999,290162130
4,0x8e6f12fcb32b19c5237e29a9fc4436ed0c8cf152,19.247789,1.052506,20.258413,281887550
5,0xb785689fe18e616fed2785eaf9b9dd5fbd11d6bd,1335.117271,1.060667,1416.114830,289169005
6,0xdebad597d42d0f2880522c8a00e00a5f6767052f,0.959798,1.048813,1.006649,281124573


In [7]:
# Function to get withdraw pool amounts
def get_withdraw_pool_amount(vault_contract: Contract):
    try:
        # Call the getWithdrawPoolAmount function
        result = vault_contract.functions.getWithdrawPoolAmount().call()
        
        # Extract scWithdrawPoolAmount and ptWithdrawPoolAmount
        scWithdrawPoolAmount = result / 1e6  # Convert to float
        
        return scWithdrawPoolAmount
    except Exception as e:
        print(f"Error fetching withdraw pool amounts: {e}")
        return None, None
    
withdrawal_pool_amount = get_withdraw_pool_amount(vault_contract)
print('Withdrawl pool', withdrawal_pool_amount)
print('Total need to withdraw', df_withdrawals['amount_usd'].sum())

amount_to_withdraw = abs(withdrawal_pool_amount - df_withdrawals['amount_usd'].sum()) #* 1.1
print('amount_to_withdraw', amount_to_withdraw, amount_to_withdraw*(1+ (0.01/100)))


Withdrawl pool 1559.081685
Total need to withdraw 6230.451902378498
amount_to_withdraw 4671.370217378498 4671.837354400235


In [8]:
data = {"action":"retry_handle_withdrawal","amount_in_usd":round(amount_to_withdraw*(1+ (0.01/100)),2),"from_addresses":None}
print(json.dumps(data))

{"action": "retry_handle_withdrawal", "amount_in_usd": 4671.84, "from_addresses": null}
